In [1]:
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import gc

In [68]:
with np.load('data/interim/kmers/kmer_matrix.npz') as data:
    kmers = data['kmers']
    kmer_order = data['kmer_order']
    genome_order = data['genome_order']

In [69]:
micsdf = joblib.load('data/interim/mic_class_dataframe2.pkl')
class_orders = joblib.load('data/interim/mic_class_order_dict2.pkl')

In [70]:
# TIO
tio_labels = class_orders['TIO']
tio_label_index = { k: v for v, k in enumerate(tio_labels) }
y_tio = np.array([ tio_label_index[m] if not pd.isna(m) else m for m in micsdf.loc[genome_order, 'TIO'] ])
labels, counts = np.unique(y_tio, return_counts=True)
ok = labels[counts >= 5]

mask = np.in1d(y_tio, ok) # Since Nan is not a label, this also filters invalid MICs
y_tio = y_tio[mask]
X_tio = kmers[mask,:]
tio_samples = genome_order[mask]
del kmers
del genome_order
del micsdf
del class_orders
gc.collect()
# %xdel kmers
# %xdel genome_order
# %xdel micsdf
# %xdel class_orders

432

In [71]:
# Test/train split
from sklearn.model_selection import train_test_split
indices = np.arange(len(tio_samples))
X_train, X_test, y_train, y_test, idx1, idx2 = train_test_split(X_tio, y_tio, indices, test_size=0.2, random_state=36, stratify=y_tio)
# del X_tio
# del y_tio
gc.collect()

0

In [6]:
print(np.unique(y_train,return_counts=True))
print(np.unique(y_test,return_counts=True))

(array([1, 2, 3, 4, 5, 6]), array([ 404, 1047,   66,    6,   78,  205]))
(array([1, 2, 3, 4, 5, 6]), array([101, 262,  16,   1,  20,  52]))


In [7]:
# Feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

fsel = SelectKBest(f_classif, k=10000)
X_train_fs = fsel.fit_transform(X_train, y_train)
X_test_fs = fsel.transform(X_test)



/home/matt/anaconda3/envs/phenores/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [  67480  149082  195348 ... 2008529 2008530 2008531] are constant.
  UserWarning)
/home/matt/anaconda3/envs/phenores/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/home/matt/anaconda3/envs/phenores/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [10]:
import tensorflow
from tensorflow import set_random_seed
set_random_seed(36)

from keras.layers.core import Dense, Dropout, Activation

from keras.models import Sequential#, load_model
from keras.utils import np_utils, to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


In [11]:
patience = 16
early_stop = EarlyStopping(monitor='loss', patience=patience, verbose=0, min_delta=0.005, mode='auto')
model_save = ModelCheckpoint("best_model.hdf5",monitor='loss', verbose = 0, save_best_only =True, save_weights_only = False, mode ='auto', period =1)
reduce_LR = ReduceLROnPlateau(monitor='loss', factor= 0.1, patience=(patience/2), verbose = 0, min_delta=0.005,mode = 'auto', cooldown=0, min_lr=0)

num_classes=6
n1 = X_test_fs.shape[1]
n2 = int((n1+num_classes)/2)
model = Sequential()
model.add(Dense(n1,activation='relu',input_dim=(n1)))
model.add(Dropout(0.5))
model.add(Dense(n2, activation='relu', kernel_initializer='uniform'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu', kernel_initializer='uniform', ))
model.add(Dense(1, kernel_initializer='normal'))

model.compile(loss='mae', metrics=['accuracy'], optimizer='adam')

In [12]:
history = model.fit(X_train_fs, y_train, epochs=100, batch_size=200, verbose=1, shuffle=True, validation_split=0.33, callbacks=[early_stop, reduce_LR])

Train on 1210 samples, validate on 596 samples
Epoch 1/100


 200/1210 [===>..........................] - ETA: 32s - loss: 2.5660 - acc: 0.0000e+00

 400/1210 [========>.....................] - ETA: 17s - loss: 2.3075 - acc: 0.0525    

 600/1210 [=============>................] - ETA: 11s - loss: 3.4434 - acc: 0.0350

 800/1210 [==================>...........] - ETA: 7s - loss: 3.1991 - acc: 0.0387 

...

 600/1210 [=============>................] - ETA: 6s - loss: 0.2679 - acc: 0.8000

 800/1210 [==================>...........] - ETA: 4s - loss: 0.2727 - acc: 0.8000

1000/1210 [=======================>......] - ETA: 2s - loss: 0.2706 - acc: 0.8050

1200/1210 [============================>.] - ETA: 0s - loss: 0.2724 - acc: 0.8058

1210/1210 [==============================] - 16s 13ms/step - loss: 0.2731 - acc: 0.8058 - val_loss: 0.3416 - val_acc: 0.7366


In [13]:
score = model.evaluate(X_test_fs, y_test, verbose=0)
print(score)
y_pred = model.predict(X_test_fs)

[0.3442011955565056, 0.7345132743362832]


In [15]:
clonaldf = pd.read_csv('data/interim/mash_population_groups.csv', index_col=0)
test_samples = tio_samples[idx2]
test_cgs = clonaldf.loc[test_samples]

resdf = pd.DataFrame(data=np.vstack((y_test, y_pred[:,0], test_cgs.values[:,0])).T,
             index=test_samples,
             columns=['true','pred','group'])
resdf['err'] = np.abs(resdf['true'] - resdf['pred'])

In [22]:
errres = resdf.groupby('group').agg(['min','max','mean','median','count'])['err']

In [24]:
errres.to_csv('tmp')

In [74]:
# Run just the worst offender

grp1_samples = clonaldf.index[clonaldf['cluster'] == 1].values
grp1_idx = np.argwhere(np.in1d(tio_samples, grp1_samples))
grp1_idx1 = idx1[np.in1d(idx1, grp1_idx)]
grp1_idx2 = idx2[np.in1d(idx2, grp1_idx)]

X_train_g1 = X_tio[grp1_idx1,:]
X_test_g1 = X_tio[grp1_idx2,:]
y_train_g1 = y_tio[grp1_idx1]
y_test_g1 = y_tio[grp1_idx2]

In [75]:
fsel = SelectKBest(f_classif, k=10000)
X_train_g1_fs = fsel.fit_transform(X_train_g1, y_train_g1)
X_test_g1_fs = fsel.transform(X_test_g1)

/home/matt/anaconda3/envs/phenores/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [    244     252     680 ... 2008547 2008548 2008549] are constant.
  UserWarning)
/home/matt/anaconda3/envs/phenores/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/home/matt/anaconda3/envs/phenores/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [128]:
patience = 16
early_stop = EarlyStopping(monitor='loss', patience=patience, verbose=0, min_delta=0.005, mode='auto')
model_save = ModelCheckpoint("best_model.hdf5",monitor='loss', verbose = 0, save_best_only =True, save_weights_only = False, mode ='auto', period =1)
reduce_LR = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=(patience/2), verbose = 0, min_delta=0.005,mode = 'auto', cooldown=0, min_lr=0)

num_classes=6
n1 = X_test_g1_fs.shape[1]
n2 = int((n1+num_classes-10)/2)
model = Sequential()
model.add(Dense(n1,activation='relu',input_dim=(n1)))
model.add(Dropout(0.5))
model.add(Dense(n2, activation='relu', kernel_initializer='uniform'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu', kernel_initializer='uniform', ))
model.add(Dense(1, kernel_initializer='normal'))

model.compile(loss='mae', metrics=['accuracy'], optimizer='adam')

In [130]:
#history = model.fit(X_train_g1_fs, y_train_g1, epochs=100, batch_size=75, verbose=1, shuffle=True, validation_split=0.33, callbacks=[early_stop, reduce_LR])
history = model.fit(X_train_g1_fs, y_train_g1, epochs=100, batch_size=50, verbose=1, shuffle=True, validation_split=0.33)

Train on 269 samples, validate on 133 samples
Epoch 1/100


 50/269 [====>.........................] - ETA: 7s - loss: 0.4802 - acc: 0.6200

100/269 [==========>...................] - ETA: 5s - loss: 0.4570 - acc: 0.6200

150/269 [===============>..............] - ETA: 3s - loss: 0.4389 - acc: 0.6533

200/269 [=====================>........] - ETA: 2s - loss: 0.4790 - acc: 0.6300

...

100/269 [==========>...................] - ETA: 5s - loss: 0.3288 - acc: 0.7800

150/269 [===============>..............] - ETA: 3s - loss: 0.3608 - acc: 0.7467

200/269 [=====================>........] - ETA: 2s - loss: 0.3758 - acc: 0.7200

250/269 [==========================>...] - ETA: 0s - loss: 0.3748 - acc: 0.7240

269/269 [==============================] - 10s 37ms/step - loss: 0.3726 - acc: 0.7212 - val_loss: 0.4415 - val_acc: 0.6541


In [132]:
y_pred_g1 = model.predict(X_test_g1_fs)
g1resdf = pd.DataFrame(data=np.vstack((y_test_g1, y_pred_g1[:,0])).T,
             columns=['true','pred'])
g1resdf['err'] = np.abs(g1resdf['true'] - g1resdf['pred'])

In [133]:
g1resdf.describe()

,true,pred,err
count,111.000000,111.000000,111.000000
mean,2.918919,2.891734,0.468760
std,1.873912,1.684550,0.317698
min,1.000000,1.432609,0.000237
25%,2.000000,1.667603,0.234079
50%,2.000000,1.825915,0.395890
75%,5.000000,5.011823,0.698466
max,6.000000,5.934915,1.352950


In [135]:
grp6_samples = clonaldf.index[clonaldf['cluster'] == 6].values
grp6_idx = np.argwhere(np.in1d(tio_samples, grp6_samples))
grp6_idx1 = idx1[np.in1d(idx1, grp6_idx)]
grp6_idx2 = idx2[np.in1d(idx2, grp6_idx)]

X_train_g6 = X_tio[grp6_idx1,:]
X_test_g6 = X_tio[grp6_idx2,:]
y_train_g6 = y_tio[grp6_idx1]
y_test_g6 = y_tio[grp6_idx2]

In [136]:
fsel = SelectKBest(f_classif, k=10000)
X_train_g6_fs = fsel.fit_transform(X_train_g6, y_train_g6)
X_test_g6_fs = fsel.transform(X_test_g6)

/home/matt/anaconda3/envs/phenores/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [     83     105     139 ... 2008547 2008548 2008549] are constant.
  UserWarning)
/home/matt/anaconda3/envs/phenores/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/home/matt/anaconda3/envs/phenores/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [139]:
patience = 16
early_stop = EarlyStopping(monitor='loss', patience=patience, verbose=0, min_delta=0.005, mode='auto')
model_save = ModelCheckpoint("best_model.hdf5",monitor='loss', verbose = 0, save_best_only =True, save_weights_only = False, mode ='auto', period =1)
reduce_LR = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=(patience/2), verbose = 0, min_delta=0.005,mode = 'auto', cooldown=0, min_lr=0)

num_classes=6
n1 = X_test_g6_fs.shape[1]
n2 = int((n1+num_classes-10)/2)
model = Sequential()
model.add(Dense(n1,activation='relu',input_dim=(n1)))
model.add(Dropout(0.5))
model.add(Dense(n2, activation='relu', kernel_initializer='uniform'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu', kernel_initializer='uniform', ))
model.add(Dense(1, kernel_initializer='normal'))

model.compile(loss='mae', metrics=['accuracy'], optimizer='adam')

In [140]:
history = model.fit(X_train_g6_fs, y_train_g6, epochs=100, batch_size=50, verbose=1, shuffle=True)

Epoch 1/100


 50/227 [=====>........................] - ETA: 1:43 - loss: 1.6907 - acc: 0.0600

100/227 [============>.................] - ETA: 40s - loss: 34.0822 - acc: 0.0300

150/227 [==================>...........] - ETA: 17s - loss: 24.8377 - acc: 0.0200

200/227 [=========================>....] - ETA: 4s - loss: 18.9944 - acc: 0.1100 

...

 50/227 [=====>........................] - ETA: 5s - loss: 0.2805 - acc: 0.8000

100/227 [============>.................] - ETA: 4s - loss: 0.3120 - acc: 0.8000

150/227 [==================>...........] - ETA: 2s - loss: 0.3068 - acc: 0.8200

200/227 [=========================>....] - ETA: 0s - loss: 0.3165 - acc: 0.8250

227/227 [==============================] - 8s 35ms/step - loss: 0.3120 - acc: 0.8238


In [141]:
y_pred_g6 = model.predict(X_test_g6_fs)
g6resdf = pd.DataFrame(data=np.vstack((y_test_g6, y_pred_g6[:,0])).T,
             columns=['true','pred'])
g6resdf['err'] = np.abs(g6resdf['true'] - g6resdf['pred'])

In [142]:
g6resdf.describe()

,true,pred,err
count,63.000000,63.000000,63.000000
mean,2.253968,1.787069,0.540105
std,1.822514,1.602802,0.519916
min,1.000000,0.834897,0.002675
25%,1.000000,0.976401,0.068940
50%,2.000000,1.040271,0.362689
75%,2.000000,1.297256,0.984276
max,6.000000,6.010312,1.872184
